In [1]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
import pickle
import keras
from keras.datasets import cifar100
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten, Lambda
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model, load_model
from keras.utils import to_categorical
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
# Training parameters
batch_size = 125
epochs = 200
num_classes = 100

data_augmentation = True
subtract_pixel_mean = True

In [3]:
# Model parameter (CIFAR-10)
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------

version = 2
n = 6

In [4]:
depth = n * 9 + 2
model_type = 'ResNet%dv%d' % (depth, version)

In [5]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train = x_train.astype(float)
x_test = x_test.astype(float)

if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

In [6]:
label_embeddings = pickle.load(open("Data/Embeddings/CIFAR/CIFAR_100_label_to_embedding_google_news.pk", "rb"))
embedding_len = len(label_embeddings[0])

y_train_embeddings = np.zeros((50000, embedding_len))
y_test_embeddings = np.zeros((10000, embedding_len))

for i in range(y_train.shape[0]):
    y_train_embeddings[i] = label_embeddings[int(y_train[i][0])]

for i in range(y_test.shape[0]):
    y_test_embeddings[i] = label_embeddings[int(y_test[i][0])]

label_embeddings_arr = np.zeros((100, embedding_len))
for i in range(100):
    label_embeddings_arr[i] = label_embeddings[i]

In [7]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [8]:
input_shape = x_train[0].shape

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
y_train shape: (50000, 100)
y_test shape: (10000, 100)


In [9]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder
    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)
    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    
    return x

In [10]:
def resnet_v2(input_shape, depth, num_classes=100, temperature=1):
    """ResNet Version 2 Model builder [b]
    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256
    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)
    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=4)(x)
    
    x = Flatten()(x)
    
    x = Dense(1024) (x)
    x = Activation('relu')(x)
    x = BatchNormalization()(x)
    
    outputs_hard = Dense(num_classes, kernel_initializer='he_normal')(x)
    outputs_hard = Lambda(lambda x: x/temperature)(outputs_hard)
    outputs_hard = Activation('softmax', name='hard')(outputs_hard)
    
    outputs_soft = Dense(embedding_len) (x)
    outputs_soft = Activation('relu')(outputs_soft)
    outputs_soft = BatchNormalization()(outputs_soft)
    outputs_soft = Dense(embedding_len, name='soft')(outputs_soft)
    
    # Instantiate model.
    model = Model(inputs=inputs, outputs=[outputs_hard, outputs_soft])

    return model

In [11]:
if data_augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0
    )
else:
    datagen = ImageDataGenerator()

In [12]:
def multi_output_generator(generator, inputs, outputs1, outputs2, batch_size=128, seed=0):
    generator1 = datagen.flow(x=inputs, y=outputs1, batch_size=batch_size, seed=seed)
    generator2 = datagen.flow(x=inputs, y=outputs2, batch_size=batch_size, seed=seed)
    while True:
        X1 = generator1.next()
        X2 = generator2.next()
        
        yield X1[0], {"hard": X1[1], "soft": X2[1]}

In [13]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
#     elif epoch > 40:
#         lr *= 5e-1
    print('Learning rate: ', lr)
    
    return lr

In [14]:
losses = {"hard": "categorical_crossentropy",
          "soft": "cosine_proximity"
         }
loss_weights = {"hard": 1.0, "soft": 2.0}
metrics={"hard": "acc"}

In [ ]:
for temperature in [50]: #, 20, 10, 5, 3, 1, 0.5, 0.1]:
    print('\n\nTemperature: %s' % (temperature))
    save_dir = os.path.join(os.getcwd(), 'saved_models/%s/combined/temp%s/' % (model_type, temperature))
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
        
    model_name = 'soft_cifar100_%s_model.{epoch:03d}.h5' % model_type
    filepath = os.path.join(save_dir, model_name)
    
    checkpoint = ModelCheckpoint(filepath=filepath,
                                 monitor='val_soft_loss',
                                 verbose=1,
                                 save_best_only=True)

    model_name = 'hard_cifar100_%s_model.{epoch:03d}.h5' % model_type
    filepath = os.path.join(save_dir, model_name)
    
    checkpoint_hard = ModelCheckpoint(filepath=filepath,
                                 monitor='val_hard_acc',
                                 verbose=1,
                                 save_best_only=True)
    
    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                                   cooldown=0,
                                   patience=5,
                                   min_lr=0.5e-6)

    callbacks = [checkpoint, checkpoint_hard, lr_reducer, lr_scheduler]
    
    
    model = resnet_v2(input_shape=input_shape, depth=depth, temperature=temperature)
    model.compile(optimizer=Adam(lr=lr_schedule(0)),
                  loss=losses, 
                  loss_weights = loss_weights, 
                  metrics=metrics)

    
    history = model.fit_generator(epochs=epochs, 
                              generator=multi_output_generator(datagen, x_train, 
                                                               y_train, y_train_embeddings, batch_size=batch_size),
                              steps_per_epoch=x_train.shape[0] // batch_size,
                              validation_data=multi_output_generator(datagen, x_test, 
                                                                     y_test, y_test_embeddings, batch_size=batch_size),
                              validation_steps = x_test.shape[0] // batch_size,
                              verbose=1, callbacks=callbacks)
    
    
    history_filepath = os.path.join(save_dir, 'cifar100_%s_w2v_history_temp.pk' % (model_type))
    pickle.dump(history, open(history_filepath, "wb"))



Temperature: 50
Learning rate:  0.001
Epoch 1/200
Learning rate:  0.001
400/400 [==============================] - 58s 145ms/step - loss: 4.1756 - hard_loss: 4.4111 - soft_loss: -0.4020 - hard_acc: 0.0374 - val_loss: 3.6366 - val_hard_loss: 4.1645 - val_soft_loss: -0.4248 - val_hard_acc: 0.0619

Epoch 00001: val_soft_loss improved from inf to -0.42475, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet56v2/combined/temp50/soft_cifar100_ResNet56v2_model.001.h5

Epoch 00001: val_hard_acc improved from -inf to 0.06190, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet56v2/combined/temp50/hard_cifar100_ResNet56v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
400/400 [==============================] - 44s 111ms/step - loss: 3.1417 - hard_loss: 3.8372 - soft_loss: -0.4812 - hard_acc: 0.1144 - val_loss: 2.9849 - val_hard_loss: 3.6818 - val_soft_loss: -0.4686 - val_hard_acc: 0.1446

Epoch 00002: val_soft_loss improved from -0.42475 to -0.46


Epoch 00014: val_soft_loss improved from -0.65401 to -0.66294, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet56v2/combined/temp50/soft_cifar100_ResNet56v2_model.014.h5

Epoch 00014: val_hard_acc did not improve from 0.51790
Epoch 15/200
Learning rate:  0.001
400/400 [==============================] - 45s 113ms/step - loss: 0.2387 - hard_loss: 1.3159 - soft_loss: -0.7052 - hard_acc: 0.6267 - val_loss: 0.6362 - val_hard_loss: 1.6570 - val_soft_loss: -0.6780 - val_hard_acc: 0.5406

Epoch 00015: val_soft_loss improved from -0.66294 to -0.67805, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet56v2/combined/temp50/soft_cifar100_ResNet56v2_model.015.h5

Epoch 00015: val_hard_acc improved from 0.51790 to 0.54060, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet56v2/combined/temp50/hard_cifar100_ResNet56v2_model.015.h5
Epoch 16/200
Learning rate:  0.001
400/400 [==============================] - 46s 115ms/step -

Epoch 30/200
Learning rate:  0.001
400/400 [==============================] - 45s 114ms/step - loss: -0.3460 - hard_loss: 0.8135 - soft_loss: -0.7757 - hard_acc: 0.7543 - val_loss: 0.4819 - val_hard_loss: 1.5143 - val_soft_loss: -0.7130 - val_hard_acc: 0.5941

Epoch 00030: val_soft_loss did not improve from -0.72477

Epoch 00030: val_hard_acc did not improve from 0.60120
Epoch 31/200
Learning rate:  0.001
400/400 [==============================] - 46s 116ms/step - loss: -0.3647 - hard_loss: 0.7974 - soft_loss: -0.7783 - hard_acc: 0.7625 - val_loss: 0.4982 - val_hard_loss: 1.5238 - val_soft_loss: -0.7110 - val_hard_acc: 0.5915

Epoch 00031: val_soft_loss did not improve from -0.72477

Epoch 00031: val_hard_acc did not improve from 0.60120
Epoch 32/200
Learning rate:  0.001
400/400 [==============================] - 46s 116ms/step - loss: -0.3856 - hard_loss: 0.7785 - soft_loss: -0.7808 - hard_acc: 0.7667 - val_loss: 0.3610 - val_hard_loss: 1.4175 - val_soft_loss: -0.7277 - val_hard_acc:

Epoch 48/200
Learning rate:  0.001
400/400 [==============================] - 46s 114ms/step - loss: -0.6776 - hard_loss: 0.5313 - soft_loss: -0.8182 - hard_acc: 0.8360 - val_loss: 0.2544 - val_hard_loss: 1.3389 - val_soft_loss: -0.7564 - val_hard_acc: 0.6520

Epoch 00048: val_soft_loss improved from -0.75504 to -0.75640, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet56v2/combined/temp50/soft_cifar100_ResNet56v2_model.048.h5

Epoch 00048: val_hard_acc improved from 0.65050 to 0.65200, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet56v2/combined/temp50/hard_cifar100_ResNet56v2_model.048.h5
Epoch 49/200
Learning rate:  0.001
400/400 [==============================] - 47s 118ms/step - loss: -0.6852 - hard_loss: 0.5248 - soft_loss: -0.8195 - hard_acc: 0.8365 - val_loss: 0.3546 - val_hard_loss: 1.4045 - val_soft_loss: -0.7402 - val_hard_acc: 0.6318

Epoch 00049: val_soft_loss did not improve from -0.75640

Epoch 00049: val_hard_acc di

400/400 [==============================] - 44s 109ms/step - loss: -0.8783 - hard_loss: 0.3681 - soft_loss: -0.8423 - hard_acc: 0.8841 - val_loss: 0.3595 - val_hard_loss: 1.4462 - val_soft_loss: -0.7627 - val_hard_acc: 0.6502

Epoch 00068: val_soft_loss did not improve from -0.76458

Epoch 00068: val_hard_acc did not improve from 0.65200
Epoch 69/200
Learning rate:  0.001
400/400 [==============================] - 44s 109ms/step - loss: -0.8838 - hard_loss: 0.3635 - soft_loss: -0.8425 - hard_acc: 0.8862 - val_loss: 0.3527 - val_hard_loss: 1.4400 - val_soft_loss: -0.7626 - val_hard_acc: 0.6494

Epoch 00069: val_soft_loss did not improve from -0.76458

Epoch 00069: val_hard_acc did not improve from 0.65200
Epoch 70/200
Learning rate:  0.001
400/400 [==============================] - 44s 109ms/step - loss: -0.8884 - hard_loss: 0.3607 - soft_loss: -0.8430 - hard_acc: 0.8855 - val_loss: 0.4207 - val_hard_loss: 1.5021 - val_soft_loss: -0.7597 - val_hard_acc: 0.6403

Epoch 00070: val_soft_loss


Epoch 00086: val_hard_acc improved from 0.71100 to 0.71420, saving model to /home/tliu/Dev/CMU/10-715/10715_Project/saved_models/ResNet56v2/combined/temp50/hard_cifar100_ResNet56v2_model.086.h5
Epoch 87/200
Learning rate:  0.0001
400/400 [==============================] - 44s 109ms/step - loss: -1.2622 - hard_loss: 0.1170 - soft_loss: -0.8961 - hard_acc: 0.9697 - val_loss: -0.0188 - val_hard_loss: 1.1741 - val_soft_loss: -0.8020 - val_hard_acc: 0.7120

Epoch 00087: val_soft_loss did not improve from -0.80616

Epoch 00087: val_hard_acc did not improve from 0.71420
Epoch 88/200
Learning rate:  0.0001
400/400 [==============================] - 44s 111ms/step - loss: -1.2765 - hard_loss: 0.1109 - soft_loss: -0.8982 - hard_acc: 0.9719 - val_loss: -0.0249 - val_hard_loss: 1.1729 - val_soft_loss: -0.8024 - val_hard_acc: 0.7147

Epoch 00088: val_soft_loss did not improve from -0.80616

Epoch 00088: val_hard_acc improved from 0.71420 to 0.71470, saving model to /home/tliu/Dev/CMU/10-715/10715_

In [ ]:
print(history.history.keys())

In [ ]:
def load_history(temperature):
    path = os.path.join(os.getcwd(), 'saved_models/%s/combined/temp%s/cifar100_%s_w2v_history_temp.pk' 
                 % (model_type, temperature, model_type))
    with open(path, 'rb') as pickle_file:
        return pickle.load(pickle_file)

In [ ]:
history50 = load_history(50)
history20 = load_history(20)
history10 = load_history(10)
history5 = load_history(5)
history3 = load_history(3)
history1 = load_history(1)
history0_5 = load_history(0.5)
history0_1 = load_history(0.1)

In [ ]:
def remove_outlier(history, key, threshold):
    arr = np.array(history.history[key])
    mask = arr > threshold
    idx = np.where(~mask, np.arange(len(mask)), 0) # array of indices, except with 0 when the value is above the threshold
    np.maximum.accumulate(idx, out=idx) # fills index 0's with max so far (i.e. the previous element)
    out = arr[idx]
    
    return out

In [ ]:
# Sometimes the validation similarity proximity spikes up for an epoch, so I just remove these values to smoothen the graph
val_soft_loss50 = remove_outlier(history50, 'val_soft_loss', -0.403)
val_soft_loss20 = remove_outlier(history20, 'val_soft_loss', -0.403)
val_soft_loss10 = remove_outlier(history10, 'val_soft_loss', -0.403)
val_soft_loss5 = remove_outlier(history5, 'val_soft_loss', -0.403)
val_soft_loss3 = remove_outlier(history3, 'val_soft_loss', -0.403)
val_soft_loss1 = remove_outlier(history1, 'val_soft_loss', -0.403)
val_soft_loss0_5 = remove_outlier(history0_5, 'val_soft_loss', -0.403)
val_soft_loss0_1 = remove_outlier(history0_1, 'val_soft_loss', -0.403)

fig=plt.figure(figsize=(16, 16))
plt.plot(val_soft_loss50[:125], label="50")
plt.plot(val_soft_loss20[:125], label="20")
plt.plot(val_soft_loss10[:125], label="10")
plt.plot(val_soft_loss5[:125], label="5")
plt.plot(val_soft_loss3[:125], label="3")
plt.plot(val_soft_loss1[:125], label="1")
plt.plot(val_soft_loss0_5[:125], label="0.5")
plt.plot(val_soft_loss0_1[:125], label="0.1")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(16, 16))
plt.plot(np.minimum.accumulate(val_soft_loss50[:125]), label="50")
plt.plot(np.minimum.accumulate(val_soft_loss20[:125]), label="20")
plt.plot(np.minimum.accumulate(val_soft_loss10[:125]), label="10")
plt.plot(np.minimum.accumulate(val_soft_loss5[:125]), label="5")
plt.plot(np.minimum.accumulate(val_soft_loss3[:125]), label="3")
plt.plot(np.minimum.accumulate(val_soft_loss1[:125]), label="1")
plt.plot(np.minimum.accumulate(val_soft_loss0_5[:125]), label="0.5")
plt.plot(np.minimum.accumulate(val_soft_loss0_1[:125]), label="0.1")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(16, 16))
plt.plot(history50.history['soft_loss'], label="50")
plt.plot(history20.history['soft_loss'], label="20")
plt.plot(history10.history['soft_loss'], label="10")
plt.plot(history5.history['soft_loss'], label="5")
plt.plot(history3.history['soft_loss'], label="3")
plt.plot(history1.history['soft_loss'], label="1")
plt.plot(history0_5.history['soft_loss'], label="0.5")
plt.plot(history0_1.history['soft_loss'], label="0.1")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(16, 16))
plt.plot(history50.history['val_hard_acc'][25:], label="50")
plt.plot(history20.history['val_hard_acc'][25:], label="20")
plt.plot(history10.history['val_hard_acc'][25:], label="10")
plt.plot(history5.history['val_hard_acc'][25:], label="5")
plt.plot(history3.history['val_hard_acc'][25:], label="3")
plt.plot(history1.history['val_hard_acc'][25:], label="1")
plt.plot(history0_5.history['val_hard_acc'][25:], label="0.5")
plt.plot(history0_1.history['val_hard_acc'][25:], label="0.1")
plt.legend()
plt.show()

In [ ]:
fig=plt.figure(figsize=(16, 16))
plt.plot(np.maximum.accumulate(history50.history['val_hard_acc'])[25:], label="50")
plt.plot(np.maximum.accumulate(history20.history['val_hard_acc'])[25:], label="20")
plt.plot(np.maximum.accumulate(history10.history['val_hard_acc'])[25:], label="10")
plt.plot(np.maximum.accumulate(history5.history['val_hard_acc'])[25:], label="5")
plt.plot(np.maximum.accumulate(history3.history['val_hard_acc'])[25:], label="3")
plt.plot(np.maximum.accumulate(history1.history['val_hard_acc'])[25:], label="1")
plt.plot(np.maximum.accumulate(history0_5.history['val_hard_acc'])[25:], label="0.5")
plt.plot(np.maximum.accumulate(history0_1.history['val_hard_acc'])[25:], label="0.1")
plt.legend()
plt.show()

In [ ]:
#Top K Predictions
def pred_top_k(y_test_pred, label_embeddings_arr=label_embeddings_arr, k=5):
    sim_table = cosine_similarity(y_test_pred, label_embeddings_arr)
    top_k_guesses = np.argpartition(sim_table,range(99-k+1,100),axis=1)[:,99-k+1:]
    return sim_table, top_k_guesses

#Top k Accuracy
def calc_top_k_acc(top_k, y_test):
    correct = 0
    for i in range(y_test.shape[0]):
        if np.squeeze(y_test)[i] in top_k[i]:
            correct += 1
    return correct/float(y_test.shape[0])

def evaluate(model, x_test, y_test, label_embeddings_arr, k=5):
    y_test_pred_hard, y_test_pred_soft = model.predict(x_test)
    sim_table, top_k_guesses = pred_top_k(y_test_pred_soft, label_embeddings_arr, k=k)

    #Top prediction
    label_predictions = sim_table.argmax(axis=1)
    
    #Accuracy
    acc = np.sum((np.squeeze(y_test) == label_predictions)) / float(y_test.shape[0])
    top_k_acc = calc_top_k_acc(top_k_guesses, y_test)
    print("Accuracy: " + str(acc))
    print("Top " + str(k) + " Accuracy: " + str(top_k_acc))
    
    return acc, top_k_acc

In [ ]:
list_acc = []
list_top_k_acc = []

k = 5
temperatures = [50, 20, 10, 5, 3, 1, 0.5, 0.1]
for temperature in temperatures:
    print("Temperature: %s" % (temperature))
    
    save_dir = os.path.join(os.getcwd(), 'saved_models/%s/combined/temp%s/' % (model_type, temperature))
    files = [f for f in os.listdir(save_dir) if f[-3:]=='.h5']
    files.sort()
    model_filepath = os.path.join(save_dir, files[-1])
    print(files[-1])
    
    model = load_model(model_filepath)
    
    acc, top_k_acc = evaluate(model, x_test, y_test, label_embeddings_arr, k=k)
    list_acc.append(acc)
    list_top_k_acc.append(top_k_acc)
    print('\n')

In [ ]:
plt.subplot(1, 2, 1)
plt.show(temperatures, list_acc)
plt.xlabel('temperature')
plt.ylabel('accuracy')
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.show(temperatures, list_acc_top_k)
plt.xlabel('temperature')
plt.ylabel('accuracy')
plt.title('Top %s Accuracy' % (k))

plt.show()

In [ ]:
# save_dir = os.path.join(os.getcwd(), 'saved_models/%s/baseline/temp%s/' % (model_type, temperature))
# model_name = 'cifar100_%s_model.{epoch:03d}.h5' % model_type
# if not os.path.isdir(save_dir):
#     os.makedirs(save_dir)
# filepath = os.path.join(save_dir, model_name)

In [ ]:
# checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_soft_loss',
#                              verbose=1,
#                              save_best_only=True)

# lr_scheduler = LearningRateScheduler(lr_schedule)

# lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                cooldown=0,
#                                patience=5,
#                                min_lr=0.5e-6)

# callbacks = [checkpoint, lr_reducer, lr_scheduler]

In [ ]:
# model = resnet_v2(input_shape=input_shape, depth=depth, temperature=temperature)
# model.compile(optimizer=Adam(lr=lr_schedule(0)),
#               loss=losses, 
#               loss_weights = loss_weights, 
#               metrics=metrics)
# model.summary()

In [ ]:
# history = model.fit_generator(epochs=epochs, 
#                               generator=multi_output_generator(datagen, x_train, 
#                                                                y_train, y_train_embeddings, batch_size=batch_size),
#                               steps_per_epoch=x_train.shape[0] // batch_size,
#                               validation_data=multi_output_generator(datagen, x_test, 
#                                                                      y_test, y_test_embeddings, batch_size=batch_size),
#                               validation_steps = x_test.shape[0] // batch_size,
#                               verbose=1, callbacks=callbacks)

In [ ]:
# history_filepath = os.path.join(save_dir, 'cifar100_%s_w2v_history_temp.pk' % (model_type))
# pickle.dump(history, open(history_filepath, "wb"))

In [ ]:
# best_model_filepath = os.path.join(save_dir, 'cifar100_%s_model.%03d.h5' % (model_type, 138))
# best_model = load_model(best_model_filepath)

# scores = best_model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])

In [ ]:
# def acc(top_k, y_true):
#     correct = 0
#     for i in range(y_true.shape[0]):
#         if y_true[i] in top_k[i]:
#             correct += 1
#     return correct/float(y_true.shape[0])

# y_pred = best_model.predict(x_test)
# print("Accuracy: %f" % (np.sum(np.argmax(y_pred, axis=1) == np.argmax(y_test, axis=1)) / float(len(y_test))))

# k = 5
# y_pred_top_k = y_pred.argsort()[:,-k:]
# print("Top %d accuracy: %f" % (k, acc(y_pred_top_k, np.argmax(y_test, axis=1))))